In [59]:
import pandas as pd
import plotly.express as px
import numpy as np

### Codificación de variables

In [60]:
datos = pd.read_csv("IcfesLimpio.csv")
datos.shape

(87884, 38)

In [61]:
# Eliminar identificador comun
datos.drop("estu_consecutivo", axis=1, inplace=True)

In [62]:
datos.nunique()

estu_tipodocumento                 11
cole_area_ubicacion                 2
cole_bilingue                       2
cole_caracter                       4
cole_cod_dane_establecimiento     310
cole_cod_dane_sede                327
cole_cod_mcpio_ubicacion          122
cole_genero                         3
cole_jornada                        6
cole_naturaleza                     2
estu_cod_depto_presentacion        19
estu_cod_mcpio_presentacion        69
estu_cod_reside_depto              18
estu_cod_reside_mcpio             195
estu_estadoinvestigacion            4
estu_genero                         2
estu_nacionalidad                   8
estu_privado_libertad               2
fami_cuartoshogar                   5
fami_educacionmadre                12
fami_educacionpadre                12
fami_estratovivienda                7
fami_personashogar                  5
fami_tieneautomovil                 2
fami_tienecomputador                2
fami_tieneinternet                  2
fami_tienela

### Variables categoricas dummy

In [63]:
# se dummifica las variables con menos de 20 categorias
lista = []
for column in datos.columns.to_list():
    if datos.nunique()[column] < 20 and column != "año":
        lista.append(column)

In [64]:
datos = pd.get_dummies(datos,columns=lista, drop_first=False)

In [65]:
datos["año_nacimiento"] = datos["estu_fechanacimiento_dt"].astype(str).str[:4].astype(int)
datos.drop("estu_fechanacimiento_dt", axis=1, inplace=True)

### codificación target encodig

In [66]:
# se utiliza target encoding para las variables de más de 20 categorias
promedio_global = datos['punt_global'].mean()
dics = {}
for column in datos.columns.to_list():
    if datos.nunique()[column] > 20 and "punt" not in column and "año" not in column:
        # Calculate mean house price for each city
        mean_encoded = datos.groupby(column)['punt_global'].mean()
        smooth_encoded = (mean_encoded * datos[column].value_counts() + promedio_global) / (datos[column].value_counts() + 1)
        datos[f'{column}_encoded'] = datos[column].map(smooth_encoded)
        coded = pd.DataFrame()
        coded[f'{column}_encoded']= datos[f'{column}_encoded']
        coded[column] = datos[column]
        dics[column] = coded
        datos.drop(column, axis=1, inplace = True)

In [67]:
datos.head()

,punt_ingles,punt_matematicas,punt_sociales_ciudadanas,punt_c_naturales,punt_lectura_critica,punt_global,año,estu_tipodocumento_CC,estu_tipodocumento_CE,estu_tipodocumento_CR,...,desemp_ingles_B1,semestre_1,semestre_2,semestre_4,año_nacimiento,cole_cod_dane_establecimiento_encoded,cole_cod_dane_sede_encoded,cole_cod_mcpio_ubicacion_encoded,estu_cod_mcpio_presentacion_encoded,estu_cod_reside_mcpio_encoded
0,53.0,62.0,53.0,51.0,54.0,274.0,2015,False,False,False,...,False,False,True,False,1999,256.552815,256.552815,253.403924,270.286881,253.708284
1,68.0,72.0,69.0,66.0,72.0,348.0,2014,False,False,False,...,False,False,True,False,1997,275.371382,273.762494,278.796528,270.286881,278.376939
2,49.0,56.0,49.0,51.0,60.0,268.0,2016,False,False,False,...,False,False,True,False,1999,271.668806,271.668806,283.446840,280.093872,283.943115
3,52.0,68.0,52.0,54.0,64.0,295.0,2016,False,False,False,...,False,False,True,False,2000,244.459308,244.459308,244.459308,254.947960,244.459308
4,40.0,59.0,28.0,45.0,51.0,227.0,2019,False,False,False,...,False,False,False,True,2002,249.823691,249.823691,273.910278,273.099522,257.853831


In [68]:
datos.to_csv("datosModelo.csv", index=False)

In [70]:
for key in dics:
    dics[key].to_csv(f"dic_{key}.csv", index=False)